# Импорт API ключей

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if OPENROUTER_API_KEY is None:
    OPENROUTER_API_KEY = "<Ваш_OpenRouter_API_ключ>"

# Инициализация клиента

In [2]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

# Получение отзывов (из файла)

In [3]:
import json
with open('reviews.json') as f:
    reviews = json.load(f)
reviews = {f'Отзыв {i+1}' : review.get('Отзыв') for i, review in enumerate(reviews)}

In [4]:
reviews

{'Отзыв 1': 'Все супер! Работает шустро. Недостатков не обнаружено, все работает исправно. Рекомендую к покупке, за свои деньги работает отлично',
 'Отзыв 2': 'За такую цену не плохо. Сдувает при малейшем ветре, и не много время тянет в право',
 'Отзыв 3': 'Хорошая комплектация(4 доп. винта, 6 защитных элементов для винтов, зап. части одним словом ). Не разу не взлетел , винты крутятся, но такое ощущение что тяги не хватает, прямо из коробки, судя по всему прислали неисправный товар, коробка была покоцанная, видимо не первый возврат. Менял аккамуляторы, заряжал их, не взлетает, думал, что датчик ругается на палас, пробовал и на ламинате и с рук и с линолиума взлетать-все одна проблема-не летает, но винты крутит.',
 'Отзыв 4': 'Классный квадрокоптер, легок в управлении, мы давольны покупкой. Все отлично, дети рады👌и это самое главное',
 'Отзыв 5': 'Сломался один из моторов. Хлипкое все. Малейший ветерок сносит. Заявленные 45 минут конечно не продержится . В лучшем случае 15 за свет 3 ма

# Используемый промпт

In [5]:
prompt = (
    "Для каждого отзыва определи его:\n"
    "\tТональность (Допустимые ответы: Положительный, Нейтральный, Отрицательный)\n"
    "\tОсновное преимущество (если преимущества нет, ответь \"Отсутствует\")\n"
    "\tОсновная проблема (если проблем нет, ответь \"Отсутствует\")\n"
    "\tОценка (значение от 1 до 5 включительно)\n\n"
    f"Отзывы: {reviews}\n"
    "Формат ответа:\n"
    "[\n"
    "\t{\n"
    "\t\t\"Номер отзыва\": number\n"
    "\t\t\"Тональность\": string\n"
    "\t\t\"Основное преимущество\": string\n"
    "\t\t\"Основная проблема\": string\n"
    "\t\t\"Оценка\": number\n"
    "\t}\n"
    "]\n"
    "ВАЖНО: Выводи ТОЛЬКО JSON-массив в формате plain text! Не добавляй никакой другой текст, объяснения, markdown, кодовые блоки вроде ```json"
)

# Тестирование модели xAI Grok 4.1 Fast

In [6]:
xAI_response = client.chat.completions.create(
    model="x-ai/grok-4.1-fast",
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0.1
)

In [7]:
json.loads(xAI_response.choices[0].message.content)

[{'Номер отзыва': 1,
  'Тональность': 'Положительный',
  'Основное преимущество': 'Работает шустро и исправно за свои деньги',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 2,
  'Тональность': 'Нейтральный',
  'Основное преимущество': 'За такую цену не плохо',
  'Основная проблема': 'Сдувает при малейшем ветре и тянет в право',
  'Оценка': 3},
 {'Номер отзыва': 3,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Хорошая комплектация',
  'Основная проблема': 'Не взлетает, тяги не хватает, неисправный товар',
  'Оценка': 1},
 {'Номер отзыва': 4,
  'Тональность': 'Положительный',
  'Основное преимущество': 'Легок в управлении, дети рады',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 5,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Отсутствует',
  'Основная проблема': 'Сломался мотор, хлипкое, мало времени полета',
  'Оценка': 1},
 {'Номер отзыва': 6,
  'Тональность': 'Отрицательный',
  'Основное преимущество

# Тестирование модели OpenAI gpt-oss-20b

In [8]:
OpenAI_response = client.chat.completions.create(
    model="openai/gpt-oss-20b:free",
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0.1
)

In [9]:
json.loads(OpenAI_response.choices[0].message.content)

[{'Номер отзыва': 1,
  'Тональность': 'Положительный',
  'Основное преимущество': 'работает шустро и исправно',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 2,
  'Тональность': 'Нейтральный',
  'Основное преимущество': 'доступная цена',
  'Основная проблема': 'сдувает при малейшем ветре и короткое время полёта',
  'Оценка': 3},
 {'Номер отзыва': 3,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'хорошая комплектация',
  'Основная проблема': 'не взлетает, недостаток тяги',
  'Оценка': 2},
 {'Номер отзыва': 4,
  'Тональность': 'Положительный',
  'Основное преимущество': 'легок в управлении, дети довольны',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 5,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Отсутствует',
  'Основная проблема': 'сломался мотор, не держит заявленное время, слабый ветер',
  'Оценка': 1},
 {'Номер отзыва': 6,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Отсутствует',


# Тестирование модели Z.AI GLM 4.5 Air

In [10]:
Z_AI_response = client.chat.completions.create(
    model="z-ai/glm-4.5-air:free",
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0.1
)

In [11]:
json.loads(Z_AI_response.choices[0].message.content)

[{'Номер отзыва': 1,
  'Тональность': 'Положительный',
  'Основное преимущество': 'Работает шустро и исправно',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 2,
  'Тональность': 'Нейтральный',
  'Основное преимущество': 'Цена',
  'Основная проблема': 'Сдувается при ветре и тянет в право',
  'Оценка': 3},
 {'Номер отзыва': 3,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Хорошая комплектация',
  'Основная проблема': 'Не взлетает, возможно неисправный товар',
  'Оценка': 1},
 {'Номер отзыва': 4,
  'Тональность': 'Положительный',
  'Основное преимущество': 'Легок в управлении, дети довольны',
  'Основная проблема': 'Отсутствует',
  'Оценка': 5},
 {'Номер отзыва': 5,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Отсутствует',
  'Основная проблема': 'Сломался мотор, хлипкое, мало времени полета',
  'Оценка': 1},
 {'Номер отзыва': 6,
  'Тональность': 'Отрицательный',
  'Основное преимущество': 'Отсутствует',
  'Основная проблема': 'Сл